In [1]:
import math
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
pd.set_option("display.max_columns", None)
from conversight import Cluster
from conversight import Dataset, Flow, Parameter, SmartAnalytics,task, ProactiveInsights, Dataset,TaskLibrary,FlowLibrary
from csContext import CSContext

Tasks loaded  !!


In [ ]:
tsk = TaskLibrary()

In [2]:
@task
def get_df(ctx: CSContext, query:str,ds_id:str)->pd.core.frame.DataFrame:
    """
    Perform this function to get the data from db
    Input : query -> sql query
            ds_id -> cs dataset id
    """
    try:
        from conversight import Dataset,Context
        #ctx=Context()
        
        ctx.log.info("fetching data from db ->")
        query = str(query)
        query = " ".join(line.strip() for line in query.splitlines())
        query = query.replace("(#)", "#").replace("(\n#\n)","#").replace("(\n#)","#").replace("(#\n)","#").replace("(#\n    )","#").replace("(\n    #)","#").replace("( #)","#").replace("(# )","#").replace("(  #)","#").replace("(#  )","#") 
        ds = Dataset(ds_id)
        df = ds.sqlDf(query)
        ctx.log.info("Dataframe was fetched successfully and its shape was {}".format(df.shape))
        
        return df
    except Exception as e:
        
        return e

In [3]:
get_df.register(libraryName = "t1")

[2024-03-22 07:52:09,130] [ERROR] This version of code is already available on version 0.1, use setVersion(0.1) from TaskLibrary to avail the code


In [4]:
tsk.reload()

Tasks loaded  !!


In [5]:
tsk.t1.get_df.promote("O")

[2024-03-22 07:52:12,354] [ERROR] Version 0.1 for the Task get_df is already in level O, these are the levels available for you to promote => [S, P, O, U]


In [ ]:
tsk.t1.get_df

In [6]:
@task
def get_parameter(ctx: CSContext, df: pd.core.frame.DataFrame, key_columns_name: str, date_column_name: str, value_column_name: str, period: int, F_Mode:str,
                 outlier_flag_by_product: bool, imputation_method: str, outlier_threshold: str, global_smoothing: bool)-> dict:
    """
    Write the doc string here, Refer Forecasting Model task
    pass outlier_threshold value as a str
    """
    try:
        import pandas as pd
        import datetime as dt
        from conversight import Dataset,Context
        #ctx=Context()
        ctx.log.info("get_parameter is started ->")
        
        fieldMapping = {"date_column":date_column_name, "value_column":value_column_name}
        threshold = [0,100]
        forecast_type = "M"
        forecast_period = period
        outlier_threshold = float(outlier_threshold)

        def impute_outliers_by_product(df, key_column, dat_column, value_column, imputation_value="median", threshold=1.5):
            ctx.log.info(f"Outliers deduction by Product method {imputation_value} with threshold {threshold}")
            def impute_outliers_group(group):
                group[value_column] = group[value_column].apply(lambda x: max(0, x))
                Q1 = group[value_column].quantile(0.25)
                Q3 = group[value_column].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - threshold * IQR
                upper_bound = Q3 + threshold * IQR
                outliers_mask = (group[value_column] < lower_bound) | (group[value_column] > upper_bound)
                if imputation_value.lower() == "mean":
                    imputed_value = group[value_column].mean()
                elif imputation_value.lower() == "median":
                    imputed_value = group[value_column].median()
                elif imputation_value.lower() == "mode":
                    imputed_value = group[value_column].mode().iloc[0]
                elif "qv" in imputation_value.lower():
                    imputation_value_qv = imputation_value.split("=")[1]
                    qv = float(imputation_value_qv)
                    qv = group[value_column].quantile(qv)
                    imputed_value = qv
                else:
                    iv = float(imputation_value)
                    imputed_value = iv
                group[value_column] = group[value_column].mask(outliers_mask, imputed_value)
                return group
            ctx.log.info("Identifying and imputing outliers..")
            df_sorted = df.sort_values(by=dat_column)
            df_imputed = df_sorted.groupby(key_column, observed=True).apply(impute_outliers_group)
            return df_imputed

        def impute_outliers(df, column_name, imputation_value="median", threshold=1.5):
            ctx.log.info(f"Outliers deduction method {imputation_value} with threshold {threshold}")
            df[column_name] = df[column_name].apply(lambda x: max(0, x))
            Q1 = df[column_name].quantile(0.25)
            Q3 = df[column_name].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            outliers = df[(df[column_name] < lower_bound) | (df[column_name] > upper_bound)]
            if imputation_value.lower() == "mean":
                imputed_value = df[column_name].mean()
            elif imputation_value.lower() == "median":
                imputed_value = df[column_name].median()
            elif imputation_value.lower() == "mode":
                imputed_value = df[column_name].mode().iloc[0]
            elif "qv" in imputation_value.lower():
                imputation_value_qv = imputation_value.split("=")[1]
                qv = float(imputation_value_qv)
                qv = df[column_name].quantile(qv)
                imputed_value = qv
            else:
                iv = float(imputation_value)
                imputed_value = iv
            ctx.log.info("Identifying and imputing outliers..")
            df[column_name] = df[column_name].mask((df[column_name] < lower_bound) | (df[column_name] > upper_bound), imputed_value)
            return df
        
        columns = key_columns_name.split(",")
        keys = "key_column"
        df[keys] = df[columns].astype(str).agg("_".join, axis=1)
        columns.append(keys)
        columns.extend([date_column_name, value_column_name])
        df = df.dropna(subset=columns).reset_index(drop=True)
    
        df = df[[fieldMapping["date_column"], fieldMapping["value_column"] ,keys]]
        df[keys] = df[keys].astype(str)
        df[fieldMapping["date_column"]] = pd.to_datetime(df[fieldMapping["date_column"]]).dt.floor("D")
        df[fieldMapping["date_column"]] =  df[fieldMapping["date_column"]].astype("datetime64")
        df = df.dropna().reset_index(drop=True)

        if F_Mode.lower() == "month":
            df[fieldMapping["date_column"]] = ((df[fieldMapping["date_column"]] + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1)).dt.floor("D"))
            df = df.groupby([fieldMapping["date_column"], keys], observed=True)[fieldMapping["value_column"]].sum().reset_index()
        elif F_Mode.lower() == "week":
            df[fieldMapping["date_column"]] = pd.to_datetime(df[fieldMapping["date_column"]]).dt.floor("D")
            df[fieldMapping["date_column"]] = df[fieldMapping["date_column"]].dt.to_period("W").apply(lambda r: r.start_time)
            df = df.groupby([fieldMapping["date_column"], keys], observed=True)[fieldMapping["value_column"]].sum().reset_index()
        df[fieldMapping["value_column"]] = df[fieldMapping["value_column"]].apply(lambda x: max(0, x))

        if F_Mode.lower() == "month":
          P = dt.datetime.today().replace(day=1).strftime("%Y-%m-%d")
          df = df[df[fieldMapping["date_column"]] < P].reset_index(drop=True)
        elif F_Mode.lower() == "week":
          current_date = dt.datetime.today()
          P = (current_date - dt.timedelta(days=current_date.weekday())).strftime("%Y-%m-%d") #current week start monday
          df = df[df[fieldMapping["date_column"]] < P].reset_index(drop=True)

        if outlier_flag_by_product and global_smoothing:
            ctx.log.error("Both variables are True, set according...")
            return {"status": "Failed", "message": "processing failed due to bad configuration"}
        else:
            if outlier_flag_by_product:
                ctx.log.info("outlier_flag_by_product is started ->")
                df_model = impute_outliers_by_product(df,keys,date_column_name,value_column_name,imputation_method,outlier_threshold)
            if global_smoothing:
                ctx.log.info("global_smoothing is started ->")
                df_model = impute_outliers(df,value_column_name,imputation_method,outlier_threshold)
        if not outlier_flag_by_product and not global_smoothing:
            df_model = df
        
        parameter ={"MAPPING":fieldMapping, "ID":keys, "CAPPING":threshold, "F_TYPE":forecast_type, "F_PERIOD":forecast_period, "DATA":df, "DATA_MODEL":df_model}
        ctx.log.info("get_parameter is ended ->")
        return parameter
    
    except Exception as e:
        return {"status": "Failed", "message": str(e)}


In [7]:
get_parameter.register(libraryName = "t1")

[2024-03-22 07:52:15,014] [ERROR] This version of code is already available on version 0.4, use setVersion(0.4) from TaskLibrary to avail the code


In [8]:
tsk.reload()

Tasks loaded  !!


In [9]:
tsk.t1.get_parameter.promote("O")

[2024-03-22 07:52:18,814] [ERROR] Version 0.4 for the Task get_parameter is already in level O, these are the levels available for you to promote => [S, P, O, U]


In [ ]:
tsk.t1.get_parameter

In [10]:
@task
def Forecasting_day(ctx: CSContext, p : dict, allow_Forecast: bool, dataset_id: str, data_level : str, table_name : str, AutoArimaTable : str, flag : str, select_model : str, F_Mode:str,
                    reRun:bool, reRunModel:str):
    """Forecasting day/week/month level"""
    try:
        import os
        import sys
        import re
        import traceback
        import pandas as pd
        import numpy as np
        import warnings
        warnings.filterwarnings("ignore")
        # import ray 
        # ray.init(ignore_reinit_error=True)
        from conversight import TaskPool
        tp = TaskPool(pool_size=480)
        import datetime as dt
        from conversight import Dataset,Context
        #ctx = Context()


        class suppress_logs(object):
            """
            A context manager for doing a "deep suppression" of stdout and stderr in
            Python, i.e. will suppress all print, even if the print originates in a
            compiled C/Fortran sub-function.
            This will not suppress raised exceptions, since exceptions are printed
            to stderr just before a script exits, and after the context manager has
            exited (at least, I think that is why it lets exceptions through).
            """
            def __init__(self):
                # Open a pair of null files
                self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
                # Save the actual stdout (1) and stderr (2) file descriptors.
                self.save_fds = [os.dup(1), os.dup(2)]
            def __enter__(self):
                # Assign the null pointers to stdout and stderr.
                os.dup2(self.null_fds[0], 1)
                os.dup2(self.null_fds[1], 2)
            def __exit__(self, *_):
                # Re-assign the real stdout/stderr back to (1) and (2)
                os.dup2(self.save_fds[0], 1)
                os.dup2(self.save_fds[1], 2)
                # Close the null files
                for fd in self.null_fds + self.save_fds:
                    os.close(fd)


        fieldMapping = p["MAPPING"]
        keys = p["ID"]
        threshold = p["CAPPING"]
        forecast_type = p["F_TYPE"]
        forecast_period = p["F_PERIOD"]
        df_act = p["DATA"]
        df_model = p["DATA_MODEL"]
        select_model = select_model.split(",")
        ctx.log.info(f"Parameters collected successfully and selected model : {select_model} and forecast mode : {F_Mode.lower()}")
        
        if F_Mode.lower() == "day":
            df_model[fieldMapping["date_column"]] = pd.to_datetime(df_model[fieldMapping["date_column"]]).dt.floor("D")
            ctx.log.info("Datetime convertion successfull")
            max_date = str(df_model[fieldMapping["date_column"]].max())
            dateprocess_max = dt.datetime.strptime(max_date, "%Y-%m-%d %H:%M:%S")
            min_date = str(df_model[fieldMapping["date_column"]].min())
            dateprocess_min = dt.datetime.strptime(min_date, "%Y-%m-%d %H:%M:%S")
            max_date_limit = (dateprocess_max - pd.DateOffset(days=1)).strftime("%Y-%m-%d")
            min_date_limit = str(dateprocess_min.year) +"-"+ str(dateprocess_min.month) +"-"+ str(dateprocess_min.day)
            
        elif F_Mode.lower() == "week":
            df_model[fieldMapping["date_column"]] = pd.to_datetime(df_model[fieldMapping["date_column"]]).dt.floor("D")
            #df_model[fieldMapping["date_column"]] = df_model[fieldMapping["date_column"]].dt.to_period("W").apply(lambda r: r.start_time)
            ctx.log.info("Datetime convertion successfull")
            max_date = str(df_model[fieldMapping["date_column"]].max())
            dateprocess_max = dt.datetime.strptime(max_date, "%Y-%m-%d %H:%M:%S")
            min_date = str(df_model[fieldMapping["date_column"]].min())
            dateprocess_min = dt.datetime.strptime(min_date, "%Y-%m-%d %H:%M:%S")
            # max_date_limit = (dateprocess_max - pd.DateOffset(weeks=1)).strftime("%Y-%m-%d")
            max_date_limit = str(dateprocess_max.year) +"-"+ str(dateprocess_max.month) +"-"+ str(dateprocess_max.day)
            min_date_limit = str(dateprocess_min.year) +"-"+ str(dateprocess_min.month) +"-"+ str(dateprocess_min.day)
                
        elif F_Mode.lower() == "month":
            df_model[fieldMapping["date_column"]] = pd.to_datetime(df_model[fieldMapping["date_column"]]).dt.floor("D")
            ctx.log.info("Datetime convertion successfull")
            max_date = str(df_model[fieldMapping["date_column"]].max())
            dateprocess_max = dt.datetime.strptime(max_date, "%Y-%m-%d %H:%M:%S")
            min_date = str(df_model[fieldMapping["date_column"]].min())
            dateprocess_min = dt.datetime.strptime(min_date, "%Y-%m-%d %H:%M:%S")
            # max_date_limit = (dateprocess_max - pd.DateOffset(months=1)).strftime("%Y-%m-01")
            max_date_limit = (dateprocess_max).strftime("%Y-%m-01")
            min_date_limit = (dateprocess_min).strftime("%Y-%m-01")
        
        ctx.log.info("Min_date_limit and Max_date_limit calculated successfully")
        ctx.log.info(f"Min - Data --> {min_date_limit} and Max - Data --> {max_date_limit}")
        
        def Heartbeat( df):
            n = pd.DataFrame(df.fcst)
            n["shift"] = n.shift(1)
            n = n.fillna(0)
            n.columns = ["ori","shift"]
            n["trend"] = n["shift"].astype(float) - n["ori"].astype(float)
            n["t"] = n["trend"]*(-1)
            n.t = np.where(n.t == -0.0, 0.0,n.t)
            return list(n["t"])

        def Percentage_change(df):
            n = pd.DataFrame(df.fcst)    
            n["s"] = n.fcst.shift(1)
            n = n.fillna(0.0)
            n["perc"] = ((n["fcst"] - n["s"])/n["s"])*100
            n = n.fillna(0.0)
            #n["perc"][0] = "initial"
            return n["perc"]
        
        def TrendCal( df):
            def linreg(X, Y):
                N = len(X)
                Sx = Sy = Sxx = Syy = Sxy = 0.0
                for x, y in zip(X, Y):
                    Sx = Sx + x
                    Sy = Sy + y
                    Sxx = Sxx + x*x
                    Syy = Syy + y*y
                    Sxy = Sxy + x*y
                det = Sxx * N - Sx * Sx
                return (Sxy * N - Sy * Sx)/det, (Sxx * Sy - Sx * Sxy)/det

            a,b = linreg(range(len(df.fcst)),df.fcst)
            x = np.arange(len(df.fcst))
            y = a*x+b
            return y
        
        def missed_data_optimized(df, keys, fieldMapping, min_date, max_date, F_Mode):
            df[fieldMapping["date_column"]] = pd.to_datetime(df[fieldMapping["date_column"]]).dt.floor("D")
            if F_Mode.lower() == "month":
                dates_required = pd.date_range(min_date, max_date, freq="MS").strftime("%Y-%m-01").tolist()
            elif F_Mode.lower() == "week":
                dates_required = pd.date_range(min_date, max_date, freq="W-MON").strftime("%Y-%m-%d").tolist()
            dates_required = pd.DataFrame(dates_required, columns=[fieldMapping["date_column"]])
            dates_required[fieldMapping["date_column"]] = pd.to_datetime(dates_required[fieldMapping["date_column"]]).dt.floor("D")
            unique_keys = df[keys].unique()
            all_combinations = pd.MultiIndex.from_product([unique_keys, dates_required[fieldMapping["date_column"]]], names=[keys, fieldMapping["date_column"]]).to_frame(index=False)
            grouped = df.groupby([keys, fieldMapping["date_column"]], observed=True)[fieldMapping["value_column"]].sum().reset_index()
            processed_data = all_combinations.merge(grouped, on=[keys, fieldMapping["date_column"]], how="left")
            processed_data[fieldMapping["value_column"]] = processed_data[fieldMapping["value_column"]].fillna(0)
            return processed_data
        
        def preprocessing(data,keys, fieldMapping,allow_forecast,min_date, max_date,F_Mode):
            limiter = 0
            if F_Mode.lower() == "day":
                if allow_forecast == True:
                    limiter = 30
                else: limiter = 180
            elif F_Mode.lower() == "week":
                if allow_forecast == True:
                    limiter = 4
                else: limiter = 24
            elif F_Mode.lower() == "month":
                if allow_forecast == True:
                    limiter = 1
                else: limiter = 6

            counts = data.groupby(keys, observed=True).size()
            valid_items = counts[counts >= limiter].index
            data = data[data[keys].isin(valid_items)]

            processed_data = missed_data_optimized(data, keys, fieldMapping, min_date, max_date, F_Mode)
            processed_data = processed_data.rename(columns={fieldMapping["date_column"]: "time"})
            return processed_data
            

        #@ray.remote
        def forecastmodel1(df,fieldMapping,prod_id,keys,forecast_period,F_Mode,df_act,parameter_df):
            warnings.simplefilter(action="ignore", category=FutureWarning)
            df = df[df[keys] == prod_id].reset_index(drop=True)
            df.drop([keys],axis = 1, inplace = True)

            df_act = df_act[df_act[keys] == prod_id].reset_index(drop=True)
            df_act.drop([keys],axis = 1, inplace = True)

            try:
                params = parameter_df[parameter_df[keys] == prod_id]
                params = params.reset_index(drop=True)
                seas_mode = params["seasonalMode"][0]
            except:
                seas_mode = "additive"#"multiplicative"

            try:
                from kats.consts import TimeSeriesData as tsd
                from kats.models.prophet import ProphetModel
                from kats.models.prophet import ProphetParams
                data = tsd(df)
                parms = ProphetParams(seasonality_mode=seas_mode)
                model = ProphetModel(data,parms)
                with suppress_logs():
                    model.fit()
                if F_Mode.lower() == "day":
                    forcast = model.predict(steps = forecast_period, freq="D")
                elif F_Mode.lower() == "week":
                    forcast = model.predict(steps = forecast_period,freq="W-MON")
                elif F_Mode.lower() == "month":
                    forcast = model.predict(steps = forecast_period,freq="MS")
            except:
                from prophet import Prophet
                df.columns = ["ds", "y"]
                m = Prophet(seasonality_mode=seas_mode) #seasonality_mode="multiplicative", scaling="minmax", seasonality_prior_scale=12
                with suppress_logs():
                    m.fit(df)
                if F_Mode.lower() == "day":
                    future = m.make_future_dataframe(periods= forecast_period,freq="D", include_history=False)
                elif F_Mode.lower() == "week":
                    future = m.make_future_dataframe(periods= forecast_period,freq="W-MON", include_history=False)
                elif F_Mode.lower() == "month":
                    future = m.make_future_dataframe(periods= forecast_period,freq="MS", include_history=False)
                forcast = m.predict(future)
                forcast = forcast[["ds", "yhat", "yhat_lower", "yhat_upper"]]
                forcast.columns = ["time","fcst","fcst_lower","fcst_upper"]
            forcast[keys] = prod_id
            forcast["actual_value"] = 0.0
            df_act.columns = ["time","actual_value"]
            df_act["fcst"] = df_act["actual_value"]
            df_act["fcst_lower"] = 0.0
            df_act["fcst_upper"] = 0.0
            df_act[keys] = prod_id  
            df_act = df_act[forcast.columns]  
            df_act = pd.concat([df_act,forcast])
            df_act["fcst"] = df_act["fcst"].round()
            df_act["fcst"] = np.where(df_act["fcst"]<0,0.0,df_act["fcst"]) 
            df_act["fcst"] = np.where(df_act["fcst"] == -0.0,0.0,df_act["fcst"]) 
            t = TrendCal(df_act)
            h = Heartbeat(df_act)
            cp = Percentage_change(df_act)
            df_act["trend"] = t
            df_act["heartbeat"] = h  
            df_act["change percentage"] =  cp
            return df_act
        
        
        #@ray.remote
        def forecastmodel2(df, fieldMapping, prod_id, keys, forecast_period,F_Mode,df_act,parameter_df):
            from pandas.tseries.offsets import DateOffset
            from statsmodels.tsa.stattools import adfuller
            import statsmodels.api as sm
            from statsmodels.tools.sm_exceptions import ConvergenceWarning
            warnings.simplefilter("ignore", ConvergenceWarning)

            df = df[df[keys] == prod_id].reset_index(drop=True)
            df.drop([keys],axis = 1, inplace = True)

            df_act = df_act[df_act[keys] == prod_id].reset_index(drop=True)
            df_act.drop([keys],axis = 1, inplace = True)
            df_act.columns = ["time",fieldMapping["value_column"]]

            def adfuller_test(sales):
                result=adfuller(sales)
                if result[1] <= 0.05:
                    return 0
                else:
                    return 1
            try:
                Seasonality_flag = adfuller_test(df[fieldMapping["value_column"]])
            except:
                Seasonality_flag = 0

            try:
                params = parameter_df[parameter_df[keys] == prod_id]
                params = params.reset_index(drop=True)
                p, q, P, Q, m, d, D = params["p"][0],params["q"][0],params["sP"][0],params["sQ"][0],params["m"][0],params["d"][0],params["sD"][0]
            except:
                def seas_len(F_Mode):
                    if F_Mode.lower().lower() == "day":
                        mm = 365
                    elif F_Mode.lower().lower() == "week":
                        mm = 52
                    elif F_Mode.lower().lower() == "month":
                        mm = 12
                    return mm
                mm = seas_len(F_Mode)
                if Seasonality_flag:
                    p, q, P, Q, m, d, D = 1, 1, 1, 1, mm, 1, 1
                else:
                    p, q, P, Q, m, d, D = 1, 1, 1, 1, mm, 0, 0

            #future_dates = [df.time[-1:] + DateOffset(months=x) for x in range(0, forecast_period + 1)]
            if F_Mode.lower().lower() == "day":
                future_dates = [df.time[-1:] + DateOffset(days=x) for x in range(0, forecast_period + 1)]
            elif F_Mode.lower().lower() == "week":
                future_dates = [df.time[-1:] + DateOffset(weeks=x) for x in range(0, forecast_period + 1)]
            elif F_Mode.lower().lower() == "month":
                future_dates = [df.time[-1:] + DateOffset(months=x) for x in range(0, forecast_period + 1)]
            future_dates = pd.DataFrame(future_dates)
            future_dates.columns = ["time"]
            future_dates = future_dates.reset_index()
            future_dates.drop(["index"], axis=1 , inplace=True)   
            future_dates.drop(future_dates.index[0:1], axis=0, inplace=True)  

            try:
                model = sm.tsa.statespace.SARIMAX(df[fieldMapping["value_column"]], order=(p, d, q),seasonal_order=(P, D, Q, m), verbose=False)
                with suppress_logs():
                    results = model.fit()
            except:
                model = sm.tsa.statespace.SARIMAX(df[fieldMapping["value_column"]], order=(p, d, q),seasonal_order=(P, D, Q, m), initialization="approximate_diffuse", verbose=False)
                with suppress_logs():
                    results = model.fit()

            future_df = pd.concat([df_act,future_dates])
            future_df = future_df.reset_index()

            future_df["forecast"] = results.predict(start = len(df), end = len(df) + forecast_period)  
            future_df[["time", fieldMapping["value_column"], "forecast"]]
            future_df[keys]  = prod_id

            future_df.fillna(0,inplace = True)
            future_df["fcst"] =   future_df["forecast"].astype(float) + future_df[fieldMapping["value_column"]].astype(float)
            future_df.drop(["forecast"],axis = 1, inplace = True) 

            future_df["fcst"] = future_df["fcst"].round()
            future_df["fcst"] = np.where(future_df["fcst"] < 0, 0.0,future_df["fcst"]) 
            future_df["fcst"] = np.where(future_df["fcst"] == -0.0, 0.0,future_df["fcst"])

            future_df["fcst_lower"] = 0.0
            future_df["fcst_upper"] = 0.0

            t = TrendCal(future_df)
            h = Heartbeat(future_df)
            cp = Percentage_change(future_df)
            future_df["trend"] = t
            future_df["heartbeat"] = h  
            future_df["change percentage"] =  cp  
            future_df["change percentage"] = future_df["change percentage"].astype(float)
            future_df = future_df.rename(columns={fieldMapping["value_column"]: "actual_value"})
            return future_df


        ##### Main forecast execution start #######################################################
        ctx.log.info("Preprocessing the Data")   
        prod_ids = df_model[keys].unique()
        ctx.log.info("Getting All unique Materials to do Forecast")
        ctx.log.info("Total unique number {}".format(len(prod_ids)))
        #df_id = ray.put(df_model)

        # frame, result = [], []
        # cnt = 1
        # for i in prod_ids:
        #     tp.execute([df_model,keys,fieldMapping, i,allow_Forecast,min_date_limit,max_date_limit,F_Mode], preprocessing)
        #     #frame.append(preprocessing.remote(df_id,keys,fieldMapping, i,allow_Forecast,min_date_limit,max_date_limit,F_Mode))
        #     if cnt % 100 == 0:
        #         result.extend(tp.collect_all()[1])
        #     if cnt%1000 == 0:
        #         ctx.log.info("Completed {} Items".format(cnt))
        #     cnt+=1
        # #result = ray.get(frame)
        # # Collect all results after the loop has finished
        # remaining_results = tp.collect_all()[1]
        # result.extend(remaining_results)

        # result = pd.concat(result)
        result = preprocessing(df_model,keys,fieldMapping,allow_Forecast,min_date_limit,max_date_limit,F_Mode)
        ctx.log.info("Forecast Preprocessing Done")
        
        prod_ids = result[keys].unique()
        ctx.log.info("Total unique number for modeling {}".format(len(prod_ids)))
        #result_id = ray.put(result)
        #df_act_id = ray.put(df_act)

        #adding missing month with actuals
        frame = result[["time", keys]]
        frame.columns = [fieldMapping["date_column"], keys]
        df_act = frame.merge(df_act, how="left", on=[fieldMapping["date_column"], keys])
        df_act = df_act.fillna(0)

        try:
            ctx.log.info("AutoArima Table name: {}".format(AutoArimaTable))
            ds = Dataset(dataset_id)
            parameter_df = ds.sqlDf(f"""select * from #{AutoArimaTable}""")
            pattern = r"^m_(.*)"  # Match "m_" at the beginning, followed by any character sequence
            parameter_df.columns = [re.sub(pattern, r"\1", col) for col in parameter_df.columns]
            ctx.log.info("Successfully collected AutoArima Parameter")
        except:
            ctx.log.info("No such {} Table in db --> Taking default parameter".format(AutoArimaTable))
            parameter_df = pd.DataFrame()

        ##Rerun Check
        if reRunModel.lower() == "model_1":
            m1reRun, m2reRun = reRun, False
            select_model = ["model_1","model_2","model_3"]
        elif reRunModel.lower() == "model_2":
            m2reRun = reRun
            select_model = ["model_2","model_3"]
        else:
            m1reRun, m2reRun = False, False

        if "Model_1" in select_model or "model_1" in select_model:
            m1prod_ids = prod_ids
            ctx.log.info("Model 1 Started ...")
            if m1reRun:
                try:
                    ctx.log.info("ReRun Model 1 Started ...")
                    reRunDf = ds.sqlDf("""select * From #reRunForecastDummy""")
                    reRunDf.columns = [re.sub(pattern, r"\1", col) for col in reRunDf.columns]
                    exclude_prod_ids = reRunDf[keys].unique()
                    m1prod_ids = list(set(m1prod_ids)-set(exclude_prod_ids))
                    ctx.log.info(f"excluded item length: {len(exclude_prod_ids)} and rerun item length: {len(m1prod_ids)}")
                except:
                    pass
                    
            frame1, cnt = [], 1
            for i in m1prod_ids:
                #frame1.append(forecastmodel1.remote(result_id,fieldMapping,i,keys,forecast_period,F_Mode,df_act,parameter_df))
                tp.execute([result,fieldMapping,i,keys,forecast_period,F_Mode,df_act,parameter_df], forecastmodel1)
                if cnt % 100 == 0:
                    frame1.extend(tp.collect_all()[1])
                if cnt%1000 == 0:
                    ctx.log.info("Completed {} Items".format(cnt))
                if cnt%2000 == 0:
                    result11 = pd.concat(frame1)
                    ds.smartAnalytics.create("reRunForecastDummy",result11,False,True,True)
                cnt+=1

            # Collect all results after the loop has finished
            remaining_results = tp.collect_all()[1]
            frame1.extend(remaining_results)
            
            #result1 = ray.get(frame1)
            result1 = pd.concat(frame1)

            if m1reRun:
                result1 = pd.concat([result1, reRunDf], ignore_index=True)

            # result1.columns = ["Forecast_Date","Forecasted_Value","Forecast_Lower","Forecast_Upper",
            #                    "Forecast_Data_Level_Key","Actual_Values","Forecast_Trend","Forecast_heartbeat","Forecast_Change"]
            result1 = result1.rename(columns={"time": "Forecast_Date", "fcst": "Forecasted_Value", "fcst_lower": "Forecast_Lower",
                                              "fcst_upper": "Forecast_Upper", keys: "Forecast_Data_Level_Key", "actual_value": "Actual_Values",
                                              "trend": "Forecast_Trend", "heartbeat": "Forecast_heartbeat", "change_percentage": "Forecast_Change"})

            result1["Forecast_Data_Level"]= data_level
            result1["Forecast_Data_Level_Key"] = result1["Forecast_Data_Level_Key"].str.strip("_")
            result1["Forecast_Method"] = "Model 1"
            result1["Forecast_Method_Version"] = "V1"
            result1["Forecasted_Flag"] = "Y"
            result1["Forcast_Explain"] = ""
              
            result1["Forecast_Change"] = 0.0
            # result1["Forecast_Change"].fillna(value=0, inplace=True)
            # result1["Forecast_Change"] = pd.to_numeric(result1["Forecast_Change"], errors="coerce")
            # result1["Forecast_Change"] = np.where(result1["Forecast_Change"] == float("-inf"),-1000.00,result1["Forecast_Change"])
            # result1["Forecast_Change"] = np.where(result1["Forecast_Change"] == float("inf"),1000.00,result1["Forecast_Change"])
            # result1["Forecast_Change"] = np.where(result1["Forecast_Change"] > 1000,1000.00,result1["Forecast_Change"])  
            # result1["Forecast_Change"] = np.where(result1["Forecast_Change"]< -1000,-1000.00,result1["Forecast_Change"])
            # #result1["Forecast_Change"] = np.clip(result1["Forecast_Change"], -1000.00, 1000.00)


            result1["Forecasted_Value"] = np.where(result1["Forecasted_Value"]<0,0,result1["Forecasted_Value"])      

            result1 = result1[["Forecast_Data_Level","Forecast_Data_Level_Key","Forecast_Method","Forecast_Method_Version","Forecasted_Flag",
                           "Forecast_Date","Actual_Values", "Forecasted_Value","Forecast_Lower", "Forecast_Upper","Forecast_Trend",
                         "Forecast_heartbeat","Forecast_Change", "Forcast_Explain"]]
            ctx.log.info("Model 1 Forecasting Done")
            ds = Dataset(dataset_id)  
            ctx.log.info("Dataset Loaded")
            response = ds.smartAnalytics.create(table_name,result1,False,flag,True)
            if "status" in response and response["status"] == "success":
                ctx.log.info("smart analytics creation is done -> {}".format(dt.datetime.utcnow()))
                ctx.log.info("Creation of table {} was done properly and smart analytics is created successfully".format(table_name))
                try:
                    ds.smartAnalytics.delete("reRunForecastDummy")
                    ctx.log.info("Successfully deleted reRunForecastDummy Table")
                except:
                    pass
            else:
                ctx.log.error("smart analytics creation is failed -> {}".format(dt.datetime.utcnow()))
                return {"status": "failed", "message": "Organization processing failed due to an error {}".format(response["message"] if "message" in response else "internalServerError")}


        if "Model_2" in select_model or "model_2" in select_model:
            m2prod_ids = prod_ids
            ctx.log.info("Model 2 Started ...")
            if m2reRun:
                try:
                    ctx.log.info("ReRun Model 2 Started ...")
                    reRunDf = ds.sqlDf("""select * From #reRunForecastDummy""")
                    reRunDf.columns = [re.sub(pattern, r"\1", col) for col in reRunDf.columns]
                    exclude_prod_ids = reRunDf[keys].unique()
                    m2prod_ids = list(set(m2prod_ids)-set(exclude_prod_ids))
                    ctx.log.info(f"excluded item length: {len(exclude_prod_ids)} and rerun item length: {len(m2prod_ids)}")
                except:
                    pass

            frame2, cnt = [], 1
            for i in m2prod_ids:
                #frame2.append(forecastmodel1.remote(result_id,fieldMapping,i,keys,forecast_period,F_Mode,df_act,parameter_df))
                tp.execute([result,fieldMapping,i,keys,forecast_period,F_Mode,df_act,parameter_df], forecastmodel2)
                if cnt % 100 == 0:
                    frame2.extend(tp.collect_all()[1])
                if cnt%1000 == 0:
                    ctx.log.info("Completed {} Items".format(cnt))
                if cnt%2000 == 0:
                    result22 = pd.concat(frame2)
                    result22.drop(["index"],axis = 1, inplace = True)
                    ds.smartAnalytics.create("reRunForecastDummy",result22,False,True,True)
                cnt+=1

            # Collect all results after the loop has finished
            remaining_results = tp.collect_all()[1]
            frame2.extend(remaining_results)
            
            #result2 = ray.get(frame2)
            result2 = pd.concat(frame2)
            result2.drop(["index"],axis = 1, inplace = True)

            if m2reRun:
                result2 = pd.concat([result2, reRunDf], ignore_index=True)

            # result2.columns = ["Forecast_Date", "Actual_Values", "Forecast_Data_Level_Key", "Forecasted_Value","Forecast_Lower","Forecast_Upper",
            #                   "Forecast_Trend","Forecast_heartbeat","Forecast_Change"]
            result2 = result2.rename(columns={"time": "Forecast_Date", "fcst": "Forecasted_Value", "fcst_lower": "Forecast_Lower",
                                              "fcst_upper": "Forecast_Upper", keys: "Forecast_Data_Level_Key", "actual_value": "Actual_Values",
                                              "trend": "Forecast_Trend", "heartbeat": "Forecast_heartbeat", "change_percentage": "Forecast_Change"})

            result2["Forecast_Data_Level"]= data_level
            result2["Forecast_Data_Level_Key"] = result2["Forecast_Data_Level_Key"].str.strip("_")
            result2["Forecast_Method"] = "Model 2"
            result2["Forecast_Method_Version"] = "V1"
            result2["Forecasted_Flag"] = "Y"
            result2["Forcast_Explain"] = ""
            result2["Actual_Values"] = result2["Actual_Values"].astype(np.float64)
            # result2["Forecast_Lower"] = 0.0
            # result2["Forecast_Upper"] = 0.0

            result2["Forecast_Change"] = 0.0
            # result2["Forecast_Change"].fillna(value=0, inplace=True)
            # result2["Forecast_Change"] = pd.to_numeric(result2["Forecast_Change"], errors="coerce")
            # result2["Forecast_Change"] = np.where(result2["Forecast_Change"] == float("-inf"),-1000.00,result2["Forecast_Change"])
            # result2["Forecast_Change"] = np.where(result2["Forecast_Change"] == float("inf"),1000.00,result2["Forecast_Change"])
            # result2["Forecast_Change"] = np.where(result2["Forecast_Change"] > 1000,1000.00,result2["Forecast_Change"])  
            # result2["Forecast_Change"] = np.where(result2["Forecast_Change"] < -1000,-1000.00,result2["Forecast_Change"])
            # # result2["Forecast_Change"] = np.clip(result2["Forecast_Change"], -1000.00, 1000.00)


            result2["Forecasted_Value"] = np.where(result2["Forecasted_Value"]<0,0,result2["Forecasted_Value"])      
            result2["Actual_Values"] = result2["Actual_Values"].astype(np.float64)
            result2 = result2[["Forecast_Data_Level","Forecast_Data_Level_Key","Forecast_Method", "Forecast_Method_Version","Forecasted_Flag",
                           "Forecast_Date","Actual_Values", "Forecasted_Value", "Forecast_Lower", "Forecast_Upper","Forecast_Trend",
                           "Forecast_heartbeat","Forecast_Change", "Forcast_Explain"]]
            
            ctx.log.info("Model 2 Forecasting Done")
            ctx.waitUntilDatasetActive(datasetIds=[dataset_id], maxRetries=4, maxWaitTime=15,incrementMultiplier=3)
            ds = Dataset(dataset_id)  
            ctx.log.info("Dataset Loaded")
            response = ds.smartAnalytics.create(table_name,result2,False,False,True)
            if "status" in response and response["status"] == "success":
                ctx.log.info("smart analytics creation is done -> {}".format(dt.datetime.utcnow()))
                ctx.log.info("Creation of table {} was done properly and smart analytics is created successfully".format(table_name))
                try:
                    ds.smartAnalytics.delete("reRunForecastDummy")
                    ctx.log.info("Successfully deleted reRunForecastDummy Table")
                except:
                    pass
            else:
                ctx.log.error("smart analytics creation is failed -> {}".format(dt.datetime.utcnow()))
                return {"status": "failed", "message": "Organization processing failed due to an error {}".format(response["message"] if "message" in response else "internalServerError")}


        if "Model_3" in select_model or "model_3" in select_model:
            ctx.log.info("Model 3 Forecasting Started")
            ds = Dataset(dataset_id)
            f = ds.sqlDf(f"select * from #{table_name}")
            columns = f.columns.str.strip("m_")
            f.columns = columns

            m1 = f[f["Forecast_Method"] == "Model 1"]
            result = f[f["Forecast_Method"] == "Model 2"]

            m1.columns = columns
            result.columns = columns

            m1 = m1[m1["Forecast_Data_Level"] == data_level]
            result = result[result["Forecast_Data_Level"] == data_level]

            result = result.sort_values(["Forecast_Data_Level_Key","Forecast_Date"], ignore_index=True)
            m1 = m1.sort_values(["Forecast_Data_Level_Key","Forecast_Date"], ignore_index=True)

            actuals = m1["Actual_Values"].tolist()
            dates = m1["Forecast_Date"].tolist()
            key = m1["Forecast_Data_Level_Key"].tolist()
            m3 = pd.DataFrame()
            fm3 = []

            for x,y in zip(m1["Forecasted_Value"],result["Forecasted_Value"]):
                z = (x+y)/2
                fm3.append(z)

            m3["fcst"] = fm3
            m3["Forecast_Data_Level_Key"] = key
            m3["Forecast_Data_Level"] = data_level
            m3["Forecast_Method"] = "Model 3"
            m3["Forecast_Method_Version"] = "V1"
            m3["Forecasted_Flag"] = "Y"
            m3["Forecast_Date"] = dates
            m3["Actual_Values"] = actuals
            m3["Forecast_Lower"] = 0.0
            m3["Forecast_Upper"] = 0.0
            m3["Forcast_Explain"] = ""
            m3["fcst"] = m3["fcst"].round()

            prod_ids = m3["Forecast_Data_Level_Key"].unique()  
            frames_of_data = pd.DataFrame()
            for i in prod_ids:
                data_frame = m3[m3["Forecast_Data_Level_Key"] == i]
                data_frame = data_frame.sort_values(["Forecast_Date"]).reset_index(drop=True)
                data_frame["Forecast_Trend"] = TrendCal(data_frame)
                data_frame["Forecast_heartbeat"] = Heartbeat(data_frame)
                data_frame["Forecast_Change"] = Percentage_change(data_frame)
                frames_of_data = pd.concat([frames_of_data,data_frame])


            frames_of_data["Forecast_Change"] = 0.0
            # frames_of_data["Forecast_Change"].fillna(value=0, inplace=True)
            # frames_of_data["Forecast_Change"] = pd.to_numeric(frames_of_data["Forecast_Change"], errors="coerce")
            # frames_of_data["Forecast_Change"] = np.where(frames_of_data["Forecast_Change"] == float("-inf"),-1000.00,frames_of_data["Forecast_Change"])
            # frames_of_data["Forecast_Change"] = np.where(frames_of_data["Forecast_Change"] == float("inf"),1000.00,frames_of_data["Forecast_Change"])
            # frames_of_data["Forecast_Change"] = np.where(frames_of_data["Forecast_Change"] > 1000,1000.00,frames_of_data["Forecast_Change"])  
            # frames_of_data["Forecast_Change"] = np.where(frames_of_data["Forecast_Change"]< -1000,-1000.00,frames_of_data["Forecast_Change"]) 

            frames_of_data=frames_of_data[["Forecast_Data_Level","Forecast_Data_Level_Key","Forecast_Method", "Forecast_Method_Version","Forecasted_Flag",
                           "Forecast_Date","Actual_Values", "fcst", "Forecast_Lower", "Forecast_Upper","Forecast_Trend",
                           "Forecast_heartbeat","Forecast_Change", "Forcast_Explain"]]

            frames_of_data.columns = ["Forecast_Data_Level","Forecast_Data_Level_Key","Forecast_Method", "Forecast_Method_Version","Forecasted_Flag",
                               "Forecast_Date","Actual_Values", "Forecasted_Value", "Forecast_Lower", "Forecast_Upper","Forecast_Trend",
                               "Forecast_heartbeat","Forecast_Change", "Forcast_Explain"]
            ctx.log.info("Model 3 Forecasting Done")
            ctx.waitUntilDatasetActive(datasetIds=[dataset_id], maxRetries=4, maxWaitTime=15,incrementMultiplier=3)
            ds = Dataset(dataset_id)
            ctx.log.info("Dataset Loaded")
            response = ds.smartAnalytics.create(table_name,frames_of_data,False,False,True)
            if "status" in response and response["status"] == "success":
                ctx.log.info("smart analytics creation is done -> {}".format(dt.datetime.utcnow()))
                ctx.log.info("Creation of table {} was done properly and smart analytics is created successfully".format(table_name))
            else:
                ctx.log.error("smart analytics creation is failed -> {}".format(dt.datetime.utcnow()))
                return {"status": "failed", "message": "Organization processing failed due to an error {}".format(response["message"] if "message" in response else "internalServerError")}

        return {"status": "success", "message": "Forecasting Done Successfully"}
    
    except Exception as e:
        #print(traceback.format_exc())
        return {"status": "Failed", "message": str(e)}


In [11]:
Forecasting_day.register(libraryName = "t1")

[2024-03-22 07:52:22,988] [INFO] Forecasting_day has been successfully registered. The most recent version available is 0.3 !!


In [12]:
tsk.reload()

Tasks loaded  !!


In [13]:
tsk.t1.Forecasting_day.promote("O")

Version 0.3 for the task Forecasting_day has been successfully promoted to O  !!
Tasks loaded  !!


In [ ]:
tsk.t1.Forecasting_day

In [14]:
flw = FlowLibrary()

Flows loaded !!


In [16]:
with Flow(name="csf_Forecast_d", description= "to do Forecast") as flow:
    query = """
    SELECT cast(m_actual_date as date) as "actual_date",
    m_sales_quantity as "sales_quantity", m_product_num as "product_num"
    FROM #FRCST_TEST
    --where cast(m_actual_date as date) < "2024-01-20"
    """
    ds_id = "662ba9e1-BZoSk0BIm"
    idp = Parameter("Dataset Id", ds_id)
    q = Parameter("Sql Query", query)
    k = Parameter("""Name of key column (the column that needs to forecasted like product id, It can also
                  be multi-level like warehouse and then product id)""","product_num")
    period = Parameter("Period (The time duration in days/weeks/months to do Forecast)",12)
    allow_for = Parameter("Allow Force Forecast", False)
    d = Parameter("Name of Date Column", "actual_date")
    v = Parameter("Name of Metric Column", "sales_quantity")
    key_name = Parameter("Name of Forecast Type Level", "ProductLevel Monthly")
    tabel_name = Parameter("Name of Forecast Table", "Latest_Forecast")
    AutoArimaTable = Parameter("Name of AutoArima Parameter Table", "AutoArima_Param")
    o = Parameter("Overwrite", True)
    select_model = Parameter("Forecast Models", "Model_1,Model_2,Model_3")
    F_Mode = Parameter("Define Forecast Mode : day/week/month", "month")
    im = Parameter("Outlier Imputation Method : mean/median/mode/quartile value(qv=0.95)", "median")
    ot = Parameter("Outlier Threshold", "1.5")
    ofp = Parameter("Outlier Imputation Flag (set yes if Product Level)(if yes then put global flag be false)", False)
    gs = Parameter("""Global Imputation Smoothing (set yes if you have atleast one off product with very high values)
                    (if yes then put outlier flag be false)""", False)
    reRun = Parameter("reRun the Flow", False)
    reRunModel = Parameter("Select Models From Where reRun Will Start: Model_1/Model_2 if reRun enabled else NA", "NA")
    
    
    a = tsk.t1.get_df(q,idp)
    get_para = tsk.t1.get_parameter(a,k,d,v,period,F_Mode,ofp,im,ot,gs)
    fore = tsk.t1.Forecasting_day(get_para,allow_for,idp,key_name,tabel_name,AutoArimaTable,o,select_model,F_Mode,reRun,reRunModel)

In [17]:
flow.register(libraryName = "Forecasting_v1",flowName ="csf_Forecast_d",description = "to do Forecast")

'The flow csf_Forecast_test has been registered successfully. Latest version available now is 0.1 !!'

In [18]:
flw.reload()

Flows loaded !!


In [19]:
flw.Forecasting_v1.csf_Forecast_d.promote("P")

'Version 0.1 for the flow csf_Forecast_test has been promoted to O successfully !!'

In [34]:
#flw.Forecasting_v1.csf_Forecast_d.deleteVersion(0.1)

version 0.1 has been deleted.
Flows loaded !!


In [ ]:
import pandas as pd

# Assuming df is your dataframe with columns "item_no", "date", "value"

# Example dataframe creation
data = {
    "item_no": [1, 1, 1, 2, 2,2,2,2,2,2,2, 3, 3, 3, 3, 4, 4],
    "date": pd.date_range("2024-01-01", periods=17, freq="D"),
    "value": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 0 ,1,1,1,1,1]
}
df = pd.DataFrame(data)

# Group by "item_no", count rows, and filter out items with less than 6 rows
counts = df.groupby("item_no").size()
valid_items = counts[counts >= 6].index

# Filter the original dataframe based on valid item numbers
filtered_df = df[df["item_no"].isin(valid_items)]

# Output the filtered dataframe
filtered_df